# `python-varname` playground

In [4]:
from varname import varname, nameof, will, config
from varname.helpers import Wrapper, register, debug

## Usage of `varname`

### Basic usage

In [5]:
def func():
    return varname()

f = func()
f

'f'